In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")



In [ ]:
from langchain_groq import ChatGroq  
llm = ChatGroq(
    model="gemma2-9b-it",  
    temperature=0.7,
    max_tokens=512
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002C79C6CD6D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002C787D8A810>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=512)

In [46]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings
    

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
# Load a single PDF file
loader = PyPDFLoader("MedicineBook.pdf")
documents = loader.load()



In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter    
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
texts

In [50]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002C84ACE81D0>, search_kwargs={'k': 2})

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define the system prompt
system_prompt = (
    "You are a doctor speaking to a patient. "
    "Use the following context to answer the question. "
    "If you don't know the answer, say you don't know.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)




{'input': 'What is the role of the liver in the human body?', 'context': [Document(id='46cf721c-03ec-454f-bc3c-eb50c675bfaf', metadata={'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'creator': 'Adobe Acrobat 6.0', 'creationdate': '2006-10-16T20:19:33+02:00', 'moddate': '2006-10-16T22:03:45+02:00', 'source': 'MedicineBook.pdf', 'total_pages': 4505, 'page': 2271, 'page_label': '2242'}, page_content='system. Damage to this system is called biliary disease.\nThe liver receives blood that comes directly from\nthe intestines. At any given time the liver contains\nabout 13% of the blood circulating in the body. This\nblood is rich in nutruents (food, vitamins, andminer-\nals) that the body needs to function. Some of the most\nimportant functions of the liver are to process these\nnutrients.\nImportant functions of the liver include:\n/C15manufacturing and regulating the production of pro-\nteins. The most important proteins made in the liver\nare albumin, which helps maintain blood volume,\nand clo

In [73]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# Dictionary to store chat histories for different sessions
store = {}

# Function to retrieve or create chat history for a session
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]




In [92]:
from langchain_core.runnables import Runnable

class WrappedRetrievalChain(Runnable):
    def __init__(self, base_chain):
        self.base_chain = base_chain

    def invoke(self, input, config=None):
        result = self.base_chain.invoke(input, config)
        return {"result": result}  # Adjust the key as needed


In [98]:
# Wrap the retrieval_chain if necessary
wrapped_chain = WrappedRetrievalChain(retrieval_chain)

# Initialize chat_chain with correct output_messages_key
chat_chain = RunnableWithMessageHistory(
    wrapped_chain,
    get_session_history,
    input_messages_key="input",
    output_messages_key="result",  # Ensure this matches the key in the output dictionary
    history_messages_key="chat_history"
)

# Define your query
query = "What are the symptoms of diabetes?"

# Invoke the chat_chain with the query and session ID
response = chat_chain.invoke(
    {"input": query},
    config={"configurable": {"session_id": "session-1"}}
)
print(response)

# Follow-up question
follow_up_query = "What was my question?"
response = chat_chain.invoke(
    {"input": follow_up_query},
    config={"configurable": {"session_id": "session-1"}}
)
print(response)


Error in RootListenersTracer.on_chain_end callback: KeyError('result')


{'result': {'input': 'What are the symptoms of diabetes?', 'chat_history': [], 'context': [Document(id='42e8a616-771d-4636-b398-60a12a2b4b13', metadata={'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'creator': 'Adobe Acrobat 6.0', 'creationdate': '2006-10-16T20:19:33+02:00', 'moddate': '2006-10-16T22:03:45+02:00', 'source': 'MedicineBook.pdf', 'total_pages': 4505, 'page': 1185, 'page_label': '1156'}, page_content='in overweight adults over the age of 40. The classic\nsymptoms include feeling tired and sick, frequent\nurination, excessive thirst, excessive hunger, and\nweight loss.\nKetoacidosis,aconditiondueto starvationoruncon-\ntrolled diabetes, is common in Type I diabetes. Ketones\nare acid compounds that form in the blood when the\nbody breaks down fats and proteins. Symptoms include\nabdominal pain, vomiting, rapid breathing, extreme\nlethargy, and drowsiness. Patients with ketoacidosis will\nalso have a sweet breath odor. Left untreated, this condi-\ntion can lead to coma and death.\n

Error in RootListenersTracer.on_chain_end callback: KeyError('result')


{'result': {'input': 'What was my question?', 'chat_history': [], 'context': [Document(id='c46e402e-235a-458c-95a7-85d55973db87', metadata={'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'creator': 'Adobe Acrobat 6.0', 'creationdate': '2006-10-16T20:19:33+02:00', 'moddate': '2006-10-16T22:03:45+02:00', 'source': 'MedicineBook.pdf', 'total_pages': 4505, 'page': 1603, 'page_label': '1574'}, page_content='/C15providing referrals to social service agencies and\ncounseling services that are LGBT-friendly\n/C15taking educational courses that are sensitive to the\nneeds of LGBT patients\n/C15treating the families of LGBT patients as one would\nthe families of heterosexual patients\n/C15maintaining the strictest code of confidentiality\n/C15developing and maintaining health care centers or\nclinics that address LGBT-specific needs\n/C15asking non-threatening questions to determine if a\nperson is at risk of an STD\n/C15educating patients of risk factors associated with\nSTDs, possible vaccines, and t